In [2]:
import sys
sys.path.append('../..')
import torch
import numpy as np
import transforms_data as td
from PIL import Image
import glob
from torch import cuda
import acquire_ear_dataset as a
import os
import shutil
from DLBio.pytorch_helpers import get_device



CATEGORIES = ["falco_len", "jesse_kru", "konrad_von", "nils_loo", "johannes_boe", "johannes_wie", "sarah_feh", "janna_qua", "tim_moe"]
CATEGORIES.sort()
AUTHORIZED = ["falco_len","konrad_von"]
RESIZE_Y = 150
RESIZE_X = 100
DATA_TEST_FOLDER = "../auth_dataset/unknown-auth/*png"
DEVICE = get_device()

model = torch.load('./class_sample/model.pt', DEVICE)

In [2]:
# Bilder aufnehmen
a.capture_ear_images(amount_pic=10, pic_per_stage=10, is_authentification=True)

In [2]:
image_array = []
files = glob.glob (DATA_TEST_FOLDER)
files.sort()
# declare function of transformation
preprocess = td.transforms_valid_and_test((RESIZE_Y, RESIZE_X),[0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

for f in files:
    image = Image.open(f)
    image_transformed = preprocess(image)
    image_transformed = image_transformed.reshape(-1, RESIZE_Y, RESIZE_X, 1)
    image_transformed = image_transformed.permute(3, 0, 1, 2)
    if cuda.is_available():
        image_array.append(image_transformed.type('torch.cuda.FloatTensor'))
    else:
        image_array.append(image_transformed.type('torch.FloatTensor'))

In [3]:
all_classes = []
summ_pred = np.empty(1)
for i in image_array:
	with torch.no_grad():
		pred = model(i)
		pred = torch.softmax(pred, 1)
		pred = pred.cpu().numpy()
		summ_pred = summ_pred + pred

	classes = np.argmax(pred, 1)
	all_classes.append(classes[0])

	pred = np.append(pred, classes)
	pred = np.append(pred, CATEGORIES[classes[0]])	
	print(pred, "\n")
print(all_classes)
print(summ_pred)

['0.08076418936252594' '0.07004870474338531' '0.08524050563573837'
 '0.21191458404064178' '0.02645527385175228' '0.423717737197876'
 '0.027206964790821075' '0.033973973244428635' '0.040678054094314575'
 '5.0' 'konrad_von'] 

['0.1117372065782547' '0.0633353665471077' '0.081693135201931'
 '0.19130420684814453' '0.027304014191031456' '0.4438782036304474'
 '0.03270541504025459' '0.021609796211123466' '0.02643265575170517' '5.0'
 'konrad_von'] 

['0.14028088748455048' '0.021559104323387146' '0.05530431121587753'
 '0.2603743374347687' '0.022174619138240814' '0.44668373465538025'
 '0.025485936552286148' '0.02009640820324421' '0.00804058276116848' '5.0'
 'konrad_von'] 

['0.17390774190425873' '0.07007075846195221' '0.08382807672023773'
 '0.16586485505104065' '0.03157679736614227' '0.362881600856781'
 '0.05234237015247345' '0.044171929359436035' '0.015355832874774933' '5.0'
 'konrad_von'] 

['0.09942641109228134' '0.053986869752407074' '0.05682184174656868'
 '0.13338468968868256' '0.0203564167

In [5]:
# Hier besser die Warscheinlichkeit über alle Bilder ermitteln und darüber prüfen.
# Beispiel: Bei 5 Bilder muss die aufsummierte Wahrscheinlichkeit für eine Person >4 sein!!

NUMBER_AUTHORIZED = int(.7*len(image_array))
authentification_dict = {CATEGORIES[i]:all_classes.count(i) for i in all_classes}
print(authentification_dict) 

for a in authentification_dict:
    if a in AUTHORIZED and summ_pred[0][CATEGORIES.index(a)]>= NUMBER_AUTHORIZED:
        print("Access granted! Welcome "  + a + "!")
        break
    else:
        print("Access denied")
        break


{'Falco': 7, 'Johannes': 1}
Access granted! Welcome Falco!


In [4]:
shutil.rmtree('../auth_dataset/unknown-auth')